In [1]:
import stockutils, basicutils, matchutils, config
import pandas_datareader.data as web, re, sqlite3, datetime

In [2]:
class StockData:
    def __init__(self, update = False):
        self.company_table = stockutils.company_table(update = update)
    
    @basicutils.memo
    def lookup(self, search_info = ('symbol', 'name', 'se'), **conditions):
        """Converts company_name to ticker symbol. Returns a tuple of (symbol, company name) 
        or returns an empty table if company or ticker is not found."""
        initial = conditions.copy()
        if isinstance(search_info, str):
            current_search = [search_info[:]]
        else:
            current_search = list(search_info)
        order = []
        if 'name' in conditions:
            fuzzy_names = []
            if not isinstance(conditions['name'], list):
                conditions['name'] = [conditions['name']] 
            for name in conditions['name']:
                fuzzy_names.extend([self.company_table[match] 
                                    for match in matchutils.fuzzy_search(name, self.company_table.keys())])
            order = conditions['name'] = fuzzy_names
            current_search.append('name')
        elif 'symbol' in conditions:
            order = conditions['symbol'] if isinstance(conditions['symbol'], list) else [conditions['symbol']]
            current_search.append('symbol')
        results = stockutils.sql_search(config.sql_path, config.sql_table_name, current_search, **conditions)
        if order:
            sorted_results = [item[:-1] for identifier in order for item in results if identifier == item[-1]]
            return sorted_results or [("No results found matching following conditions: ", str(initial))]
        return results or [("No results found matching following conditions: ", str(initial))]

            
    
            
#       def stock_price(self, ticker, date):
            
        
        

In [3]:
basicutils.import_cache('data/cached_searches.json')

In [4]:
basicutils.clear_cache('data/cached_searches.json')

In [5]:
w = StockData()

In [6]:
w.lookup(name='Apple')

[('AAPL', 'Apple Inc.', 'nasdaq'),
 ('APLE', 'Apple Hospitality REIT, Inc.', 'nyse')]

In [7]:
w.lookup(name = 'Nvidia',se = 'amex')

[('No results found matching following conditions: ',
  "{'name': 'Nvidia', 'se': 'amex'}")]

In [8]:
w.lookup(ipoyear=(list(range(1990,1991))), se = 'amex')

[('IF', 'Aberdeen Indonesia Fund, Inc.', 'amex')]

In [9]:
w.lookup(name = "Dick's")

[('DKS', "Dick's Sporting Goods Inc", 'nyse')]

In [10]:
w.lookup(name = 'Becton', se = 'nyse')

[('BDX', 'Becton, Dickinson and Company', 'nyse')]

In [11]:
w.lookup(name = ['Intel', 'BD'])

[('INTC', 'Intel Corporation', 'nasdaq'),
 ('ADRD', 'BLDRS Developed Markets 100 ADR Index Fund', 'nasdaq'),
 ('CBD', 'Companhia Brasileira de Distribuicao', 'nyse'),
 ('LND', 'Brasilagro Cia Brasileira De Propriedades Agricolas', 'nyse'),
 ('BLIN          ', 'Bridgeline Digital, Inc.', 'nasdaq'),
 ('DSU', 'Blackrock Debt Strategies Fund, Inc.', 'nyse'),
 ('BDX', 'Becton, Dickinson and Company', 'nyse'),
 ('BDE', 'Black Diamond, Inc.', 'nasdaq'),
 ('BCH', 'Banco De Chile', 'nyse'),
 ('SBS',
  'Companhia de saneamento Basico Do Estado De Sao Paulo - Sabesp',
  'nyse'),
 ('BHL', 'Blackrock Defined Opportunity Credit Trust', 'nyse'),
 ('DTLA^', 'Brookfield DTLA Inc.', 'nyse')]

In [12]:
w.lookup(name = 'Intel Corporation', ipoyear = '15')

[('No results found matching following conditions: ',
  "{'ipoyear': '15', 'name': 'Intel Corporation'}")]

In [13]:
w.lookup(search_info = ['marketcap', 'name'], symbol = 'BDX')

[('$38.42B', 'Becton, Dickinson and Company')]

In [14]:
w.lookup(name ='Becton Dickinson and Company')

[('BDX', 'Becton, Dickinson and Company', 'nyse')]

In [15]:
basicutils.json_dump('data/cached_searches.json')

In [16]:
w.lookup(name = 'Mgmt')

[('ARES', 'Ares Management L.P.', 'nyse'),
 ('WM', 'Waste Management, Inc.', 'nyse'),
 ('MDLY', 'Medley Management Inc.', 'nyse'),
 ('OMAA', 'OM Asset Management plc', 'nyse'),
 ('RM', 'Regional Management Corp.', 'nyse'),
 ('MMAC', 'MMA Capital Management, LLC', 'nasdaq'),
 ('NLY', 'Annaly Capital Management Inc', 'nyse'),
 ('APO', 'Apollo Global Management, LLC', 'nyse'),
 ('CLMS', 'Calamos Asset Management, Inc.', 'nasdaq'),
 ('EEQ', 'Enbridge Energy Management LLC', 'nyse')]